In [1]:
import xarray as xr
import xroms
from glob import glob

# How to load data

This demonstrates how to read in ROMS model output from thredds, a list of netCDF files, and a list of zarr files.

Chunks are used to break up model output into smaller units for use with `dask`. Inputting chunks into a call to open a dataset requires the use of `dask`. This can be formalized more by setting up a `dask` cluster. The best sizing of chunks is not clear *a priori* and requires some testing.

A big benefit to reading in model output using one of these functions is that it adds `z` coordinates and other useful metrics to the `Dataset`, including the z coordinates on each horizontal grid (e.g., z_rho_u), and the z coordinates relative to mean sea level (e.g., z_rho0). It also sets up an xgcm grid object for the Dataset, which is stored in each DataArray in the Dataset in its attributes, and is accessible with the accessor (`ds.xroms.grid`).

## single netCDF location

Model output stored in netCDF format with one address (local file location or thredds server address `loc`) can be read in with 

    xroms.open_netcdf(loc, chunks).

In [2]:
# Forecast model output of the Texas-Louisiana shelf
loc = 'http://barataria.tamu.edu:8080/thredds/dodsC/forecast_latest/txla2_his_f_latest.nc'

# A basic chunk choice
chunks = {'ocean_time':1}

# no special xroms function required to read in from thredds
ds = xroms.open_netcdf(loc, chunks=chunks)

## list of netCDF files

For a list of local netCDF files, with their `.nc` suffix, containing ROMS output:

    ds = xroms.open_netcdf(files, chunks)

Some default selections are automatically chosen for reading in the files:

    {'compat': 'override', 'combine': 'by_coords',
         'data_vars': 'minimal', 'coords': 'minimal', 'parallel': True}
         
Any of these can be overwritten by inputting a dictionary with updated entries. Other desired inputs to `xr.open_mfdataset` can also be input this way.

    xrargs = {'parallel': False}

In [3]:
files = glob('../xroms/tests/input/ocean_his_000?.nc')
ds = xroms.open_mfnetcdf(files, chunks=chunks)

## zarr files example

For a list of zarr output files, with no suffix, containing ROMS output, use:

    ds = xroms.open_zarr(files, chunks)
    
Some default selections are automatically chosen for reading in the files:

    {'consolidated': True, 'drop_variables': 'dstart'}
    
and for concatenating the files together:

    {'dim': 'ocean_time', 'data_vars': 'minimal', 'coords': 'minimal'}
         
Any of these can be overwritten by inputting a dictionary called `xrargs` with updated entries. Other desired inputs to `xr.open_zarr` can also be input this way. To modify or add arguments to `concat`, use `xrconcatargs`.

We can use test model output to demonstrate. We use `chunks={'ocean_time':2}` because there are two files, each with 2 time steps.

In [4]:
files = glob('../xroms/tests/input/ocean_his_000?')
ds = xroms.open_zarr(files, chunks = {'ocean_time':2})

## Save output

After model output has been read in with `xarray`, it can be used for calculations and/or subsetted, then easily saved back out to a file:

    ds.isel(ocean_time=0).to_netcdf('filename.nc')